# Introduction:
- This is a project that makes use of publicly a ... 


In [ ]:
# Dependency for calendar holidays for canada.
pip install holidays

In [ ]:
# Import relevant libs
import numpy as np
import pandas as pd
import dateutil
import re

# Just change the following two lines to match your environment path!
dir = '/Users/fei/Library/CloudStorage/OneDrive-Personal/Documents/RIT/BANA 680 Data Management for Business Analytics/Assignment 2/'
bwt_data = 'bwt-taf-2010-2014-eng.csv'

# Data Frames creation
df = pd.read_csv(dir+bwt_data)
df.head()

,CBSA Office,Location,Updated,Commercial Flow,Travellers Flow
0,St. Stephen,"St. Stephen, NB",2014-04-04 12:58 ADT,Not Applicable,No Delay
1,St. Stephen,"St. Stephen, NB",2014-04-04 11:55 ADT,Not Applicable,No Delay
2,St. Stephen,"St. Stephen, NB",2014-04-04 10:46 ADT,Not Applicable,No Delay
3,St. Stephen,"St. Stephen, NB",2014-04-04 10:14 ADT,Not Applicable,No Delay
4,St. Stephen,"St. Stephen, NB",2014-04-04 09:02 ADT,Not Applicable,No Delay


# Data Cleanup of aggregated data in dataset

In [ ]:
# We only want data from Lewiston Bridge crossing for now
indexFiltered = df[df['CBSA Office'] != "Queenston-Lewiston Bridge"].index
df.drop(indexFiltered, axis =0,  inplace=True)
print(df.shape) # Shape before filtering
df.head()

(44234, 5)


,CBSA Office,Location,Updated,Commercial Flow,Travellers Flow
369413,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 13:06 EDT,No Delay,No Delay
369414,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 12:05 EDT,No Delay,No Delay
369415,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:09 EDT,No Delay,No Delay
369416,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07 EDT,No Delay,No Delay
369417,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07 EDT,No Delay,No Delay


In [ ]:
print(df.shape) # Shape before filtering

from dateutil import parser
# Datetime cleanup; Make Date field a Pandas date field with UTC time as standard 
# Maybe there is a more efficient method, but works for now - but takes a while to process...
tzmapping = {'ADT': dateutil.tz.gettz('Canada/Atlantic'),
            'CDT': dateutil.tz.gettz('US/Central'),
            'MDT': dateutil.tz.gettz('America/Denver'),
            'PDT': dateutil.tz.gettz('US/Pacific')}
df['Updated'] = df['Updated'].apply(parser.parse, tzinfos=tzmapping)

# Create a copy of the df for Logistic regression where we drop the no delay and other irrelevant cases
df1 = df.copy(deep=True) # let's do a deep copy to not interfere in results
indexFiltered = df1[(df['Travellers Flow'] == "Not Applicable") | 
        (df['Travellers Flow'] == "Closed")].index
df1.drop(indexFiltered, axis =0,  inplace=True)
            
df1.loc[(df['Travellers Flow'] == "No Delay"), 'Travellers Flow'] = 0
# df1.loc[df['Travellers Flow'] == "Delay", 'Travellers Flow'] = 1

# Change values for the text values on the Travellers flow columns
df.loc[(df['Travellers Flow'] == "No Delay") |
        (df['Travellers Flow'] == "Not Applicable") | 
        (df['Travellers Flow'] == "Closed"), 'Travellers Flow'] = 0

# Check if it is clean: returns nothing is good!
df.loc[(df['Travellers Flow'] == "No Delay") |
        (df['Travellers Flow'] == "Not Applicable") | 
        (df['Travellers Flow'] == "Closed")]

# df after cleanup to check
print(df.shape) # Shape before filtering
print(df1.shape) # Shape before filtering

# df['Updated'] = pd.to_datetime(df['Updated'], utc=True)
df.head()


# We have 2 dfs for the different regression analysis we want to do
# For OLS - df and for Logistic - df1


(44234, 5)
(44234, 5)
(44222, 5)


,CBSA Office,Location,Updated,Commercial Flow,Travellers Flow
369413,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 13:06:00-04:00,No Delay,0
369414,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 12:05:00-04:00,No Delay,0
369415,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:09:00-04:00,No Delay,0
369416,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07:00-04:00,No Delay,0
369417,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07:00-04:00,No Delay,0


In [ ]:
# Add date column to both dfs
df['Date'] = pd.to_datetime(df['Updated']).dt.date
df1['Date'] = pd.to_datetime(df1['Updated']).dt.date

# Add Weekday vs weekend check 
df["IsWeekend"] = pd.to_datetime(df["Date"]).dt.weekday >= 5
df1["IsWeekend"] = pd.to_datetime(df1["Date"]).dt.weekday >= 5

df1.head()

,CBSA Office,Location,Updated,Commercial Flow,Travellers Flow,Date,IsWeekend
369413,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 13:06:00-04:00,No Delay,0,2014-04-04,False
369414,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 12:05:00-04:00,No Delay,0,2014-04-04,False
369415,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:09:00-04:00,No Delay,0,2014-04-04,False
369416,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07:00-04:00,No Delay,0,2014-04-04,False
369417,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07:00-04:00,No Delay,0,2014-04-04,False


In [ ]:
# To check if we have weekend true vs false:
# df.loc[(df['IsWeekend'] == False)] # no output needed

In [ ]:
# Let's add a column for CANADA Bank holiday checks

# from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import holidays

cal = calendar()
df['Date'] = pd.to_datetime(df['Date']).dt.normalize()

canada_holidays = holidays.Canada()

holidays = cal.holidays(df['Date'].min(), df['Date'].max())

# df.dtypes
df['isHoliday'] = df['Date'].isin(canada_holidays)
df.head()

,CBSA Office,Location,Updated,Commercial Flow,Travellers Flow,Date,IsWeekend,isHoliday
369413,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 13:06:00-04:00,No Delay,0,2014-04-04,False,False
369414,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 12:05:00-04:00,No Delay,0,2014-04-04,False,False
369415,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:09:00-04:00,No Delay,0,2014-04-04,False,False
369416,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07:00-04:00,No Delay,0,2014-04-04,False,False
369417,Queenston-Lewiston Bridge,"Queenston, ON",2014-04-04 11:07:00-04:00,No Delay,0,2014-04-04,False,False


In [ ]:
# To check if we have public holiday true vs false:
# df.loc[(df['isHoliday'] == True)]

# OLS Regression:

# Logistic regression:

# Additional Data Analysis

# Conclusions